### Part 1:

Plan - modify my solution from day 12 but couldn't get that. Liberally read through other python solution in the reddit AoC thread.

Super helpful resources:
- https://github.com/SourishS17/aoc2022/blob/main/day16a.py (also a video explaining process)
- https://stackabuse.com/courses/graphs-in-python-theory-and-implementation/lessons/depth-first-search-dfs-algorithm/

Important Learnings:
- any 0 valve slows us down: we can bypass them (they slow us down by 1 instead of 2 min since we have to take a minute to get through them prior to moving to another)
- once a valve is open it released `p` every minute. This was not initially clear!
- the DFS approach I copied is able to handle backtracking since it is always looking at all neighbors: 
    - Assume I have `AA` connected to `BB,CC` and `BB` connected to `EE`.
    - At `AA` I will consider the following:
        - opening to valve `AA` (a one-time step)
        - moving to `BB`
        - moving to `CC`
    - Let's say I move to `BB`, then my options are:
        - open valve `BB` (assuming not already open)
        - move to `EE` 
        - move to `AA` -> we could keep bouncing back and forth, of course we will also take all other routes to 30 in this case.

In [1]:
import numpy as np
from collections import deque

with open("data/day16.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
 
# ugly
valve_flow = {l.split(' ')[1]: int(l.split('rate=')[1].split(';')[0]) for l in lines}
valve_graph = {l.split(' ')[1]: l.replace('valves ', 'valve ').split('valve ')[1].split(',') for l in lines}
# darn white spaces...
valve_graph = {k: [vv.strip() for vv in v] for k,v in valve_graph.items()}


In [2]:
# minutes, nodes opened, start, total
q=deque([(0,(),"AA",0)])
visited=set() # track visited nodes
max_pressure =0 # global max value

# while there are elements in our queue
while q:
    
    # pop of each component
    minutes,opened_valves,current,pressure=q.popleft()
    
    # if hit 30 min then check if we 
    # have exceeded current max (w)
    if minutes==30: 
        max_pressure=max(max_pressure,pressure)
        continue
        
    # don't revisit IF the opened valves & current
    # have been visited.
    # This means we'd be revisiting just to close
    # a valve from the same position we entered
    # waste of time -> not super clear on this
    if (opened_valves,current) in visited:
        continue
    
    # add to our visited set
    visited.add((opened_valves,current))
    
    # We iterate through the opened valves since a minute has passed
    # and sum up their pressure onto our prior pressure passed forward
    pp = pressure
    for i in opened_valves:
        pp += valve_flow[i]
        
    # we will only open our current valve
    # if it is not 0. Otherwise this is a waste of time
    if valve_flow[current]!=0:
        
        # assuming we haven't opened, then we will 
        # add to our opened valve list
        # and we will increment by 1 more minute
        if current not in opened_valves:
            q.append((minutes+1,tuple(list(opened_valves)+[current]),current,pp))
            
    # Iterate through all nodes connected to current
    # This is great as it will also allow us to backtrack along a graph
    # And open unopened valves connected on separate
    # lineage branches (for lack of a better term)
    for i in valve_graph[current]:
        q.append((minutes+1,opened_valves,i,pp))

print(max_pressure)

2077


### Part 2:

Thoughts before reviewing another solution:
- The goal is to optimize turning on as many valves as possible, considering the elephant's progress.
- It seems like we could write the above to share some universal queue....but I think this also explodes in size because for every `1 of N` paths I am on there are `N` paths to consider from the elephant.
- Can we just run it twice and find all possible sets, and from there find the most distinct with the largest value? ugly but doable...

In [8]:
import numpy as np
from collections import deque

with open("data/day16_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
 
# ugly
valve_flow = {l.split(' ')[1]: int(l.split('rate=')[1].split(';')[0]) for l in lines}
valve_graph = {l.split(' ')[1]: l.replace('valves ', 'valve ').split('valve ')[1].split(',') for l in lines}
# darn white spaces...
valve_graph = {k: [vv.strip() for vv in v] for k,v in valve_graph.items()}


In [20]:
# minutes, nodes opened, start, total
q=deque([(0,(),"AA",0)])
visited=set() # track visited nodes
max_pressure =0 # global max value
elf_paths = set()

# while there are elements in our queue
while q:
    
    # pop of each component
    minutes,opened_valves,current,pressure=q.popleft()
    
    # if hit 30 min then check if we 
    # have exceeded current max (w)
    if minutes==30: 
        elf_paths.add((opened_valves, pressure))
        continue
        
    # don't revisit IF the opened valves & current
    # have been visited.
    # This means we'd be revisiting just to close
    # a valve from the same position we entered
    # waste of time -> not super clear on this
    if (opened_valves,current) in visited:
        continue
    
    # add to our visited set
    visited.add((opened_valves,current))
    
    # We iterate through the opened valves since a minute has passed
    # and sum up their pressure onto our prior pressure passed forward
    pp = pressure
    for i in opened_valves:
        pp += valve_flow[i]
        
    # we will only open our current valve
    # if it is not 0. Otherwise this is a waste of time
    if valve_flow[current]!=0:
        
        # assuming we haven't opened, then we will 
        # add to our opened valve list
        # and we will increment by 1 more minute
        if current not in opened_valves:
            q.append((minutes+1,tuple(list(opened_valves)+[current]),current,pp))
            
    # Iterate through all nodes connected to current
    # This is great as it will also allow us to backtrack along a graph
    # And open unopened valves connected on separate
    # lineage branches (for lack of a better term)
    for i in valve_graph[current]:
        q.append((minutes+1,opened_valves,i,pp))

print(f"Total elf paths: {len(elf_paths)}")

Total elf paths: 679


In [21]:
# we can now build all sets...which again is going to explode I am guessing
# I used a set which reduced size a lot...
import itertools

for i in itertools.product(elf_paths, elf_paths):
    if len(set(i[0][0]).intersection(set(i[1][0]))) == 0:
        print(i[0][1], i[1][1])